# Severity
#### 이전연구에서 사용했던 potential varibales는 다음과 같습니다.
*SAPS II \
*APACHE \
*MIMIC-IV 는 더 다양할 수 있습니다.

In [2]:
#potential tables
#mimiciv_derived, charlson, mai_admin, None, False, False, False, False
#mimiciv_derived, apsiii, mai_admin, None, False, False, False, False
#mimiciv_derived, oasis, mai_admin, None, False, False, False, False
#mimiciv_derived, sapsii, mai_admin, None, False, False, False, False
#mimiciv_derived, sofa, mai_admin, None, False, False, False, False
#mimiciv_derived, first_day_sofa, mai_admin, None, False, False, False, False
#mimiciv_derived, sepsis3, mai_admin, None, False, False, False, False
#https://www.mdcalc.com/calc/3917/charlson-comorbidity-index-cci

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [4]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


*saps ii (derived), dasis(derived), apsiii(derived), oasis(derived)

In [49]:
#DB 에서 데이터 가져오기

# curs.execute("select * from mimiciv_derived.charlson;")
# columns_name = [desc[0] for desc in curs.description]
# charlson = pd.DataFrame(curs.fetchall(), columns=columns_name) 

# curs.execute("select * from mimiciv_derived.apsiii;")
# columns_name = [desc[0] for desc in curs.description]
# apsiii = pd.DataFrame(curs.fetchall(), columns=columns_name) 

# curs.execute("select * from mimiciv_derived.oasis;")
# columns_name = [desc[0] for desc in curs.description]
# oasis = pd.DataFrame(curs.fetchall(), columns=columns_name) 

# curs.execute("select * from mimiciv_derived.sapsii;")
# columns_name = [desc[0] for desc in curs.description]
# sapsii = pd.DataFrame(curs.fetchall(), columns=columns_name) 

# curs.execute("select * from mimiciv_derived.sofa;")
# columns_name = [desc[0] for desc in curs.description]
# sofa = pd.DataFrame(curs.fetchall(), columns=columns_name) 

# curs.execute("select * from mimiciv_derived.sepsis3;")
# columns_name = [desc[0] for desc in curs.description]
# sepsis3 = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [17]:
# charlson.to_csv('../outputs/charlson.csv', index=False, encoding='utf-8-sig')
# apsiii.to_csv('../outputs/apsiii.csv', index=False, encoding='utf-8-sig')
# oasis.to_csv('../outputs/oasis.csv', index=False, encoding='utf-8-sig')
# sapsii.to_csv('../outputs/sapsii.csv', index=False, encoding='utf-8-sig')
# sofa.to_csv('../outputs/sofa.csv', index=False, encoding='utf-8-sig')
# sepsis3.to_csv('../outputs/sepsis3.csv', index=False, encoding='utf-8-sig')

In [2]:
charlson = pd.read_csv('../outputs/charlson.csv')
apsiii = pd.read_csv('../outputs/apsiii.csv')
oasis = pd.read_csv('../outputs/oasis.csv')
sapsii = pd.read_csv('../outputs/sapsii.csv', parse_dates=['starttime','endtime'])
sofa = pd.read_csv('../outputs/sofa.csv', parse_dates=['starttime','endtime'])
sepsis3 = pd.read_csv('../outputs/sepsis3.csv', parse_dates=['antibiotic_time','culture_time','suspected_infection_time','sofa_time'])

In [19]:
#charlson, apsiii, oasis 는 입원 시 측정하는 점수인듯
#aps iii 는 APACHE-III 이전에 생리학검사 점수임. => 계산할 필요가 있을지?https://github.com/MIT-LCP/mimic-code/issues/653
#oasis는 그대로 사용
#sapsii 도 그대로 사용
#sofa는 spo2/fio2 ratio 인듯? pao2fio2ratio_novent	pao2fio2ratio_vent meta정보가 많은데 왜 많은지 봐야할듯.
#sepsis3 는 deathcosality 와 연관해서 볼 정보같고, severity 점수로 학습에 포함하지는 않을 것.


### 각 정보 환자정보와 join

In [3]:
subjectlist1 = pd.read_csv('./data/sub1list.csv', parse_dates=['admittime','dischtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  6725 non-null   int64         
 1   subject_id  6725 non-null   int64         
 2   hadm_id     6725 non-null   int64         
 3   stay_id     6724 non-null   float64       
 4   admittime   6725 non-null   datetime64[ns]
 5   dischtime   6725 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3)
memory usage: 315.4 KB


In [4]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [5]:
sub1_charlson = sub1list >> left_join(charlson, by=['subject_id','hadm_id']) >> select('subject_id','hadm_id','stay_id','charlson_comorbidity_index')

In [6]:
sub1_charlson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   subject_id                  6725 non-null   int64  
 1   hadm_id                     6725 non-null   int64  
 2   stay_id                     6724 non-null   float64
 3   charlson_comorbidity_index  6725 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 210.3 KB


In [7]:
apsiii.head(3)

,subject_id,hadm_id,stay_id,apsiii,apsiii_prob,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,12466550,23998182,30000153,44,0.086546,7.0,6.0,0.0,0.0,NaN,...,0.0,0.0,5.0,7.0,0.0,NaN,NaN,0.0,1.0,15.0
1,13180007,27543152,30000213,44,0.086546,0.0,7.0,0.0,6.0,NaN,...,0.0,7.0,0.0,11.0,2.0,0.0,0.0,5.0,NaN,3.0
2,18421337,22413411,30000484,79,0.331267,1.0,15.0,2.0,8.0,NaN,...,5.0,0.0,5.0,11.0,0.0,0.0,0.0,0.0,NaN,29.0


In [8]:
sub1_apsiii = sub1list >> left_join(apsiii, by=['subject_id','hadm_id','stay_id']) >> select('subject_id','hadm_id','stay_id','apsiii')

In [9]:
sub1_apsiii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  6725 non-null   int64  
 1   hadm_id     6725 non-null   int64  
 2   stay_id     6724 non-null   float64
 3   apsiii      6724 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 210.3 KB


In [10]:
sub1_oasis = sub1list >> left_join(oasis, by = ['subject_id','hadm_id','stay_id']) >> select('subject_id','hadm_id','stay_id','stay_id','oasis')

In [11]:
sub1_oasis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  6725 non-null   int64  
 1   hadm_id     6725 non-null   int64  
 2   stay_id     6724 non-null   float64
 3   oasis       6724 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 210.3 KB


In [12]:
sub1_sapsii = sub1list >> left_join(sapsii, by = ['subject_id','hadm_id','stay_id']) >> select('subject_id','hadm_id','stay_id','starttime','endtime','sapsii')


In [13]:
sub1_sapsii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6725 entries, 0 to 6724
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   subject_id  6725 non-null   int64         
 1   hadm_id     6725 non-null   int64         
 2   stay_id     6724 non-null   float64       
 3   starttime   6724 non-null   datetime64[ns]
 4   endtime     6724 non-null   datetime64[ns]
 5   sapsii      6724 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 315.4 KB


In [28]:
sub1_sofa = sub1list >> left_join(sofa, by = ['stay_id']) >> select('subject_id','hadm_id','stay_id','starttime','endtime','pao2fio2ratio_vent','sofa_24hours')

In [29]:
# null 값 확인 = 2개 확인

print('NULL COUNT:')
print(len(sub1_sofa[sub1_sofa.sofa_24hours.isna()] ))  # starttime, endtime도 없기 때문에 제거하겠음.
sub1_sofa = sub1_sofa[sub1_sofa.sofa_24hours.notna()] 

NULL COUNT:
2


In [30]:
# 중복치 확인 -> 없음
## 중복치 기준: stay_id 내에서 starttime, endtime이 동일한 경우

duplicates = sub1_sofa[sub1_sofa.duplicated(subset=['stay_id', 'starttime', 'endtime'], keep=False)]
duplicates_sorted = duplicates.sort_values(by=['stay_id', 'starttime', 'endtime'])   # optional

duplicates_sorted

,subject_id,hadm_id,stay_id,starttime,endtime,pao2fio2ratio_vent,sofa_24hours


In [31]:
# hadm_id, stay_id 별로 시간순 (charttime) 정렬 (Optional)
sub1_sofa.sort_values(by=['subject_id', 'hadm_id', 'starttime', 'endtime'], inplace=True)
sub1_sofa

,subject_id,hadm_id,stay_id,starttime,endtime,pao2fio2ratio_vent,sofa_24hours
0,10001884,26184834,37510196.0,2131-01-11 04:00:00,2131-01-11 05:00:00,NaN,3.0
1,10001884,26184834,37510196.0,2131-01-11 05:00:00,2131-01-11 06:00:00,NaN,3.0
2,10001884,26184834,37510196.0,2131-01-11 06:00:00,2131-01-11 07:00:00,NaN,3.0
3,10001884,26184834,37510196.0,2131-01-11 07:00:00,2131-01-11 08:00:00,NaN,3.0
4,10001884,26184834,37510196.0,2131-01-11 08:00:00,2131-01-11 09:00:00,NaN,3.0
...,...,...,...,...,...,...,...
1324038,19999840,21033226,38978960.0,2164-09-17 09:00:00,2164-09-17 10:00:00,NaN,7.0
1324039,19999840,21033226,38978960.0,2164-09-17 10:00:00,2164-09-17 11:00:00,NaN,7.0
1324040,19999840,21033226,38978960.0,2164-09-17 11:00:00,2164-09-17 12:00:00,NaN,7.0
1324041,19999840,21033226,38978960.0,2164-09-17 12:00:00,2164-09-17 13:00:00,NaN,7.0


In [32]:
sub1_sofa.count()

subject_id            1324041
hadm_id               1324041
stay_id               1324041
starttime             1324041
endtime               1324041
pao2fio2ratio_vent      69458
sofa_24hours          1324041
dtype: int64

In [33]:
sub1_charlson.to_csv('../outputs/sub1_charlson.csv', index=False, encoding='utf-8-sig')
sub1_apsiii.to_csv('../outputs/sub1_apsiii.csv', index=False, encoding='utf-8-sig')
sub1_oasis.to_csv('../outputs/sub1_oasis.csv', index=False, encoding='utf-8-sig')
sub1_sapsii.to_csv('../outputs/sub1_sapsii.csv', index=False, encoding='utf-8-sig')
sub1_sofa.to_csv('../outputs/sub1_sofa.csv', index=False, encoding='utf-8-sig')